# Build the Neural Network

In [1]:
import os
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [6]:
device = torch.accelerator.current_accelerator().type \
    if torch.accelerator.is_available()\
    else "cpu"

print(f"Using {device} device")

Using cpu device


In [7]:
# Define the Class

class NeuralNetwork(nn.Module) :
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Prediction class: {y_pred}")

Prediction class: tensor([5])


In [13]:
# Model Layers

input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [14]:
# nn.Flatten

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [15]:
# nn.Linear

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [16]:
# nn.ReLU

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2128,  0.2004, -0.3483,  0.6846,  0.4108,  0.1166, -0.4169,  0.0513,
          0.2619, -0.0276,  0.0501,  0.5867,  0.6822,  0.0231,  0.0602,  0.1241,
          0.2605,  0.0312, -0.1252,  0.2983],
        [ 0.5083,  0.4643, -0.3414,  0.7173,  0.4247, -0.0266,  0.1733, -0.0949,
         -0.0215,  0.0358,  0.2984,  0.6796,  0.2259,  0.4669,  0.0223, -0.2946,
          0.4808,  0.0572, -0.2458,  0.4151],
        [ 0.2109,  0.4082, -0.2344,  0.2606,  0.5565,  0.1508,  0.0632, -0.1754,
         -0.0387, -0.2566, -0.1218,  0.6848,  0.2060, -0.2639,  0.1156,  0.0965,
          0.2654, -0.0469, -0.3990,  0.3058]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2128, 0.2004, 0.0000, 0.6846, 0.4108, 0.1166, 0.0000, 0.0513, 0.2619,
         0.0000, 0.0501, 0.5867, 0.6822, 0.0231, 0.0602, 0.1241, 0.2605, 0.0312,
         0.0000, 0.2983],
        [0.5083, 0.4643, 0.0000, 0.7173, 0.4247, 0.0000, 0.1733, 0.0000, 0.0000,
         0.0358, 0.2984, 0.6796, 0.2259, 0.4669, 0.02

In [17]:
# nn.Sequential

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [18]:
# nn.Softmax

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [21]:
# Model Parameters

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0077, -0.0037, -0.0020,  ...,  0.0055, -0.0005, -0.0210],
        [-0.0347,  0.0014,  0.0003,  ...,  0.0236,  0.0084,  0.0107]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0191, -0.0120], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0320, -0.0203,  0.0223,  ...,  0.0255,  0.0049, -0.0039],
        [ 0.0236, -0.0347,  0.0109,  ...,  0.0317, -0.0059,  0.0119]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 